# Description

__Цель__

Хотим понять места в приложении, которые непонятны пользователю и провоцируют его уход без регистрации.

__Задачи__

1. Собрать необходимые данные
2. Подготовить данные для аналитики
3. Провести анализ
    1. Построить сводные таблицы
    2. Визуализировать траектории
    3. Построить классификатор
        1. Помогает выделить специфичные для конкретной группы траектории
        2. Позволяет по текущей траектории оценить вероятность ухода пользователя, например, чтобы динамически менять контент приложения, чтобы люди оставались

__Ожидаемые результаты__

1. Узнать наиболее проблемные участки приложения, чтобы предотвратить уход юзеров
2. Получить классификатор, который позволит предсказывать уход пользователя по тому, как он использует приложение

# Download data

In [ ]:
import retentioneering
from retentioneering.utils import download_events, download_events_multi, preparing

Firstly, we need to load a google cloud credentials.

In [ ]:
client, job_config = retentioneering.init_from_json('./settings.json')
settings = retentioneering.Config('./settings.json')

Now we can execute needed query:

* user_filter_event_names -- takes only users who had that event
* dates_users -- dates where user_filter_event_names was happen
* users_app_version -- filter on version
* event_filter_event_names -- events in our interest
* dates_events -- period of analysis
* count_events -- count of first / last events for user

In [ ]:
# df = download_events(
#     client,
#     job_config=job_config,
#     user_filter_event_names=[u'first_open'],
#     users_app_version='7.4.2', 
#     event_filter_event_names=[u'screen_view',
#                               u'myFlights_add',
#                               u'myFlights_edit',
#                               u'myFlights_refresh',
#                               u'profile_edit_close',
#                               u'tabbar_select_page',
#                               u'welcome_see_screen',
#                               u'feed_widget_present',
#                               u'welcome_login_google',
#                               u'welcome_login_tripit',
#                               u'welcome__loginFailure',
#                               u'feed_ad_canBePresented',
#                               u'myFlights_connectEmail',
#                               u'myFlights_swipe_action',
#                               u'newFlight_myflights_see',
#                               u'welcome__chooseLoginType',
#                               u'welcome_otherLogin__show',
#                               u'newFlight_awardwallet_see',
#                               u'welcome_otherLogin__close',
#                               u'welcome_login_google_cancel',
#                               u'welcome_privacy_policyShown',
#                               u'welcome_privacy_policyShown',
#                               u'welcome_privacy_policyDecline',
#                               u'welcome_privacy_policyDecline',
#                               u'welcome_privacy_policyAccepted',
#                               u'welcome_privacy_policyAccepted',
#                               u'welcome_privacy_policyTapToPolicy',
#                               u'feed_widget_aircraft_amenities_saw',
#                               u'welcome_otherLogin__chooseLoginType',
#                               u'feed_widget_aircraft_noAircraftImage',
#                               u'welcome_otherLogin_privacy_policyShown',
#                               u'welcome_otherLogin_privacy_policyShown',
#                               u'welcome_otherLogin_privacy_policyDecline',
#                               u'welcome_otherLogin_privacy_policyDecline',
#                               u'welcome_otherLogin_privacy_policyAccepted'], 
#     dates_users=(u'2018-10-01', u'2018-10-01'), 
#     dates_events=(u'2018-10-01', u'2018-10-01'), 
#     count_events=40, 
#     return_dataframe=True
# )

Or we can put all of it in `settings['sql']` (you can see example in current directory) and execute query with it

In [ ]:
df = download_events_multi(client, job_config=job_config, settings=settings)
print 'Downloaded DataFrame shape: {}'.format(df.shape)

#### Prepare your dataset for further analysis

In [ ]:
# select target users from settings['users']
print 'Started DataFrame shape: {}'.format(df.shape)
df = preparing.filter_users(df, settings=settings)
print 'DataFrame shape after user filters: {}'.format(df.shape)

# delete events from settings['events']
df = preparing.filter_events(df, settings=settings)
print 'DataFrame shape after event filters: {}'.format(df.shape)

# drop duplicated events hapenning during settings['events']['duplicate_thr_time']
df = preparing.drop_duplicated_events(df, settings=settings)
print 'DataFrame shape after drop duplicated events: {}'.format(df.shape)

# add passed events from settings['positive_event']
df = preparing.add_passed_event(df, settings=settings)
print 'DataFrame shape after adding passed events: {}'.format(df.shape)

# add lost events from settings['negative_event']
df = preparing.add_lost_events(df, settings=settings)
print 'DataFrame shape after adding lost events: {}'.format(df.shape)

#### Look on first 5 records in prepared dataset

In [ ]:
df.head()

#### Save DataFrame if needed

In [ ]:
# choose your path
path = '../../data/data_from_bq.csv'
df.to_csv(path, sep=';', index=False)

# Analysis

Now we are ready for analysis

In [ ]:
import pandas as pd
path = '../../data_from_bq.csv'
df = pd.read_csv(path, sep=';')

## Ad-hoc

In [ ]:
from retentioneering import analysis

#### Pivot tables of event distribution by user steps

In [ ]:
desc = analysis.get_desc_table(df, settings=settings, plot=True)

По горизонтале отображен порядковый номер ивента в траектории.

По вертикали -- события приложения.

В ячейках отображается доля события на конкретном шаге.

1. В таблице видно, что много юзеров уходит сразу же после того, как увидели welcome_see_screen. Это может возникать потому что пользователь ожидал, что сможет сразу начать использовать приложение без регистрации, либо он не увидел желаемый способ авторизации.
2. Гипотеза про логин также поддерживается тем, что люди открывают другие способы авторизации

По агрегированной таблице сложно сделать более детальный анализ, поэтому разделим выборку на тех, кто ушел сразу и прошел дальше.

We can split data for `lost` and `passed` users to compare behaviour 

In [ ]:
lost_users_list = df[df.event_name == 'lost'].user_pseudo_id
filt = df.user_pseudo_id.isin(lost_users_list)
df_lost = df[filt]
df_passed = df[~filt]

desc_loss = analysis.get_desc_table(df_lost, settings, plot=True)

Теперь становится понятно, что доля пользователь, которые уходят сильно выше, чем 15%.
Судя по всему, довольно большая доля пользователей уходит на 6-7 шаге, потому что у них возникают проблемы с логином.

In [ ]:
desc_passed = analysis.get_desc_table(df_passed,  settings, plot=True)

And plot heatmap of differences

In [ ]:
diff_df = analysis.get_diff(desc_loss, desc_passed, settings=settings, precalc=True)

Здесь мы видим, что пользователи, которые регистрируются и пользуются дальше приложением сразу же пытаются посмотреть способ логина, прочитать условия использования, чаще с ними соглашаются и в итоге просто логинятся через аккаунт Google.

We can get aggregates over edges

In [ ]:
agg_list = ['trans_count', 'dt_mean', 'dt_median', 'dt_min', 'dt_max']
df_agg = analysis.get_all_agg(df, agg_list)
df_agg.head()

Здесь мы можем увидеть какие переходы занимают больше всего времени и как часто люди переходят из одного события в другое.

Например, выведем 10 в среднем самых долгих переходов.

In [ ]:
df_agg.sort_values('dt_mean', ascending=False).head(10)

Тут также можно увидеть места, на которых юзеры задерживаются больше всего (разумно смотреть только на те переходы, где достаточно большое количество переходов, потому что такие значения более устойчивы).

Например, мы видим, что логин через аккаунт Google занимает достаточно большое время, скорее всего, связано с тем, чтобы вспомнить пароль.

Люди долго читают условия использования и тд.

And adjacency matrix from it

In [ ]:
adj_count = analysis.get_adjacency(df_agg, 'trans_count')
adj_count

Также можно построить простую кластеризацию по частотности событий

In [ ]:
countmap = analysis.utils.plot_frequency_map(df, settings)

In [ ]:
analysis.utils.plot_clusters(df, countmap, n_clusters=5, plot_cnt=2)

Мы видим, что по частотность можно разделить выборку на кластеры, которые однозначно отличаются друг от друга по распределению интересующих нас событий. Визуализация траекторий будет в разделе про предиктивную модель.

# Graph visualization

You can visualize your graph in python

In [ ]:
analysis.utils.plot_graph_python(df_agg, 'trans_count', settings)

Or with our api

`Api sends aggregated graph to our server for visualization`

In [ ]:
from retentioneering.utils.export import plot_graph_api
plot_graph_api(df_lost, settings)

# Lost classifier

Fit the model

In [ ]:
clf = analysis.Model(df, target_event='lost', settings=settings)
clf.fit_model()

You can get a simple access to your quality metrics

In [ ]:
print 'ROC-AUC: {:.2f}'.format(clf.average_precision_score)
print 'PR-AUC: {:.2f}'.format(clf.roc_auc_score)

You can predict probabilities for a certain user, even they was not in train data

In [ ]:
# first we need to aggregate events by a user
data = analysis.prepare_dataset(df, target_event='lost')
# now we can predict probability for her track
vec = clf._get_vectors(data.event_name.iloc[:1])
clf.predict_proba(vec)

You can visualize tsne projection of events vs targets

In [ ]:
clf.plot_projections()

Or vs probability from model

In [ ]:
clf.plot_projections(sample=data.event_name.values, ids=data.user_pseudo_id.values)

You can select intrested cluster with bbox and visualize track for it

In [ ]:
# write coordinates bbox angles

bbox = [
    [-4, -12],
    [8, -26]
]

clf.plot_cluster_track(bbox)

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=[10, 10])
plt.scatter(clf._cached_tsne[:, 0], clf._cached_tsne[:, 1], c=clf.target)
plt.grid()
plt.title('TSNE over Tf-Idf transform of user tracks')

You can highlight major nodes and edges with our api